In [3]:
pip install pyspark

/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import pyarrow.parquet as pq
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import explode, col, lit
from pyspark.sql.functions import month, year, sum as spark_sum
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pyspark'

In [10]:
spark = SparkSession.builder \
    .appName("Stack Parquet Files") \
    .getOrCreate()

file_paths = [
    "D:/BDP_Datasets/fhvhv_tripdata_2021-01.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-02.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-03.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-04.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-05.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-06.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-07.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-08.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-09.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-10.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-11.parquet",
    "D:/BDP_Datasets/fhvhv_tripdata_2021-12.parquet"
]

dfs = []
for file_path in file_paths:
    df = spark.read.parquet(file_path)
    dfs.append(df)

stacked_df = dfs[0]
for df in dfs[1:]:
    stacked_df = stacked_df.union(df)

stacked_df = stacked_df.withColumn("airport_fee", when(stacked_df["airport_fee"].isNull(), 0).otherwise(stacked_df["airport_fee"]))
stacked_df.show(5)

main_df = stacked_df.select(
    "hvfhs_license_num",
    "pickup_datetime",
    "dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "trip_miles",
    "trip_time",
    "base_passenger_fare",
    "tolls",
    "bcf",
    "sales_tax",
    "congestion_surcharge",
    "airport_fee",
    "tips",
    "driver_pay"
)
main_df.show(5)

main_df = main_df.na.fill(0, subset=[
    'base_passenger_fare',
    'tolls',
    'bcf',
    'sales_tax',
    'congestion_surcharge',
    'airport_fee',
    'tips'
])

# Create the new column 'total_price' by summing up the required columns
main_df = main_df.withColumn('total_price', 
                             col('base_passenger_fare') +
                             col('tolls') +
                             col('bcf') +
                             col('sales_tax') +
                             col('congestion_surcharge') +
                             col('airport_fee') +
                             col('tips'))

comp_code = {
    "HV0003": "Uber",
    "HV0004": "Via",
    "HV0005": "Lyft"
}

df_uber = main_df.filter(main_df['hvfhs_license_num'] == 'HV0003')
df_via = main_df.filter(main_df['hvfhs_license_num'] == 'HV0004')
df_lyft = main_df.filter(main_df['hvfhs_license_num'] == 'HV0005')

df_uber.show(5)
df_via.show(5)
df_lyft.show(5)


#spark.stop()

c:\users\shashwat\appdata\local\programs\python\python37\lib\site-packages\pyspark\context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [11]:
# Start Spark session
"""spark = SparkSession.builder \
    .appName("WeatherDataProcessing") \
    .getOrCreate()"""

# Read the CSV file into a PySpark DataFrame
weather_df = spark.read.csv("E:/BDP_Datasets/nyc 2021-01-01 to 2021-12-31.csv", header=True, inferSchema=True)

# Show the first few rows of the weather DataFrame
weather_df.show(5)

# Remove specified columns from the DataFrame
columns_to_drop = ["name", "address", "resolvedAddress", "severerisk"]
weather_df = weather_df.drop(*columns_to_drop)

# Replace null values in the windgust column with 0
weather_df = weather_df.fillna({'windgust': 0})

# Show the modified DataFrame
weather_df.show(5)

# Stop Spark session
#spark.stop()


+----+-------+--------------------+----------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+-------+----------+
|name|address|     resolvedAddress|  datetime|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|uvindex|severerisk|
+----+-------+--------------------+----------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+-------+----------+
| nyc|    nyc|New York, NY, Uni...|2021-01-01| 2.5|     -0.2|-3.0|    67.8| 15.33|       100|      rain| 0.0|      0.0|    null|     15.5|   69.8|          1028.9|      50.6|      14.0|      3|      null|
| nyc|    nyc|New York, NY, Uni...|2021-01-02| 5.8|      3.6| 1.2|    74.0|  2.38|       100|      rain| 1.9|      0.6|    54.6|     25.5|  246.9|          1012.4|      63.9|      

In [13]:
data = df_uber

In [15]:
type(data)

pyspark.sql.dataframe.DataFrame

In [17]:
from pyspark.sql.functions import col, unix_timestamp

data = data.withColumn('pickup_timestamp', unix_timestamp('pickup_datetime'))
data = data.withColumn('dropoff_timestamp', unix_timestamp('dropoff_datetime'))
data = data.withColumn('trip_duration', (col('dropoff_timestamp') - col('pickup_timestamp')) / 60)
data = data.withColumn('driver_pay_per_mile', col('driver_pay') / col('trip_miles'))
data = data.withColumn('driver_pay_per_minute', col('driver_pay') / col('trip_duration'))

data.show()

+-----------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+------------------+-------------------+----------------+-----------------+------------------+---------------------+
|hvfhs_license_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|       total_price|driver_pay_per_mile|pickup_timestamp|dropoff_timestamp|     trip_duration|driver_pay_per_minute|
+-----------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+------------------+-------------------+----------------+-----------------+------------------+---------------------+
|           HV0003|2021-01-01 00:33:44|2021-01-01 00:49:07|     

In [20]:
data.columns

['hvfhs_license_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'base_passenger_fare',
 'tolls',
 'bcf',
 'sales_tax',
 'congestion_surcharge',
 'airport_fee',
 'tips',
 'driver_pay',
 'total_price',
 'driver_pay_per_mile',
 'pickup_timestamp',
 'dropoff_timestamp',
 'trip_duration',
 'driver_pay_per_minute']

In [21]:
weather_df.columns

['datetime',
 'temp',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipprob',
 'preciptype',
 'snow',
 'snowdepth',
 'windgust',
 'windspeed',
 'winddir',
 'sealevelpressure',
 'cloudcover',
 'visibility',
 'uvindex']

In [22]:
weather_df = weather_df.withColumnRenamed('datetime', 'pickup_datetime')
merged_df = data.join(weather_df, 'pickup_datetime', 'inner')
merged_df.show()

+-------------------+-----------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+------------------+-------------------+----------------+-----------------+------------------+---------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+-------+
|    pickup_datetime|hvfhs_license_num|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|       total_price|driver_pay_per_mile|pickup_timestamp|dropoff_timestamp|     trip_duration|driver_pay_per_minute|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|uvindex|
+-------------------+-----------------+-------------------+------------+----------

In [23]:
merged_df.columns

['pickup_datetime',
 'hvfhs_license_num',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'base_passenger_fare',
 'tolls',
 'bcf',
 'sales_tax',
 'congestion_surcharge',
 'airport_fee',
 'tips',
 'driver_pay',
 'total_price',
 'driver_pay_per_mile',
 'pickup_timestamp',
 'dropoff_timestamp',
 'trip_duration',
 'driver_pay_per_minute',
 'temp',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipprob',
 'preciptype',
 'snow',
 'snowdepth',
 'windgust',
 'windspeed',
 'winddir',
 'sealevelpressure',
 'cloudcover',
 'visibility',
 'uvindex']

### EDA